In [4]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sb

from matplotlib.pyplot import rcParams

import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
plt.style.use('seaborn-whitegrid')